# Joining Data

You have the tools to obtain data from a single table in whatever format you want. But what if the data you want is spread across multiple tables?

That's where the **JOIN** comes in! **JOIN** is incredibly important in practical SQL workflows. 

## Example

We'll use our imaginary `pets` table, which has three columns:

- `ID` - ID number of the pet
- `Name` - name of the pet
- `Animal` - type of animal

We'll also add another table, called `owners`. This table also has three columns:

- `ID` - ID number for the owner
- `Name` - name of the owner
- `Pet_ID` - ID number for the pet that belongs to the owner (which matches the ID number for the pet in the `pets` table)

To get information that applies to a certain pet, we match the `ID` column in the `pets` table to the `Pet_ID` column in the `owners` table.

For example:

- the `pets` table shows that Dr. Harris Bonkers is the pet with ID 1

- the `owners` table shows that Aubrey Little is the owner of the pet with ID 1

Putting these two facts together, Dr. Harris Bonkers is owned by Aubrey Little

![alt text](petsJoin.PNG "pets join")

Fortunately, we don't have to do this by hand to figure out which owner goes with which pet. In the next section, you'll learn how to use **JOIN** to create a new table combining information from the `pets` and `owners` tables.

## JOIN

Using **JOIN**, we can write a query to create a table with just two columns: the name of the pet and the name of the owner:

    SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
    FROM `bigquery-public-data.pet_records.pets` AS p 
    INNER_JOIN `bigquery-public-data.pet_records.owners` AS o
    ON p.ID = o.Pet_ID

We combine information from both tables by matching rows where the `ID` column in the `pets` table matches the `Pet_ID` column in the `owners` table.

In the query, **ON** determines which column in each table to use to combine the tables. Notice that since the `ID` column exists in both tables, we have to clarify which one to use. We use `p.ID` to refer to the `ID` column from the `pets` table, and `o.Pet_ID` refers to the `Pet_ID` column from the `owners` table.

    In general when you're joining tables, it's a good habit to specify which table each of your columns come from. That way, you don't have to pull up the schema every time you go back to read the query
    
The type of **JOIN** we're using today is called an **INNER_JOIN**. That means that a row will only be put in the final output table if the value in the columns you're using to combine them shows up in both the tables you're joining. For example, if Tom's ID number of 4 didn't exist in the `pets` table, we would only get 3 rows back from this query. There are other types of **JOIN**, but an **INNER JOIN** is very widely used, so it's a good one to start with.

## Example: How many files are covered by each type of software license?

GitHub is the most popular place to collaborate on software projects. A GitHub repository (or repo) is a collection of files associated with a specific project.

Most repos on GitHub are shared under a specific legal license, which determines the legal restrictions on how they are used. For our example, we're going to look at how many different files have been released under each license.

We'll work with two tables in the database. The first table is the `licenses` table, which provides the name of each GitHub repo (in the repo_name column) and its corresponding license. Here's a view of the first five rows.

In [1]:
# SETUP
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/levka/Downloads/KaggleSQL-79493a7efc0a.json"
from google.cloud import bigquery
%matplotlib inline

In [2]:
client = bigquery.Client()

db_ref = client.dataset("github_repos", project = "bigquery-public-data")
db = client.get_dataset(db_ref)
tb_ref = db.table("licenses")
licenses = client.get_table(tb_ref)
client.list_rows(licenses, max_results = 5).to_dataframe()

,repo_name,license
0,azuredream/chat_server-client,artistic-2.0
1,Egyptian19/JemCraft,artistic-2.0
2,ZioRiP/cookie,artistic-2.0
3,ajs/perl6-log,artistic-2.0
4,JohanPotgieter/Internet,artistic-2.0


The second table is the `sample_files` table, which provides, among other information, the GitHub repo that each file belongs to (in the repo_name column). The first several rows of this table are printed below.

In [3]:
tb_ref = db.table("sample_files")
sample_files = client.get_table(tb_ref)
client.list_rows(sample_files, max_results = 5).to_dataframe()

,repo_name,ref,path,mode,id,symlink_target
0,git/git,refs/heads/master,RelNotes,40960,62615ffa4e97803da96aefbc798ab50f949a8db7,Documentation/RelNotes/2.10.0.txt
1,np/ling,refs/heads/master,tests/success/plug_compose.t/plug_compose.ll,40960,0c1605e4b447158085656487dc477f7670c4bac1,../../../fixtures/all/plug_compose.ll
2,np/ling,refs/heads/master,fixtures/strict-par-success/parallel_assoc_lef...,40960,b59bff84ec03d12fabd3b51a27ed7e39a180097e,../all/parallel_assoc_left.ll
3,np/ling,refs/heads/master,fixtures/sequence/parallel_assoc_2tensor2_left.ll,40960,f29523e3fb65702d99478e429eac6f801f32152b,../all/parallel_assoc_2tensor2_left.ll
4,np/ling,refs/heads/master,fixtures/success/my_dual.ll,40960,38a3af095088f90dfc956cb990e893909c3ab286,../all/my_dual.ll


Next, we write a query that uses information in both tables to determine how many files are released in each license.

In [4]:
# Query to determine the number of files per license, sorted by number of files
query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
file_count_by_license = query_job.to_dataframe()

In [5]:
file_count_by_license

,license,number_of_files
0,mit,20624490
1,gpl-2.0,17119548
2,apache-2.0,7232387
3,gpl-3.0,4960665
4,bsd-3-clause,2944149
5,agpl-3.0,1309345
6,lgpl-2.1,800782
7,bsd-2-clause,700697
8,lgpl-3.0,567987
9,mpl-2.0,468534


It's a big query, and so we'll investigate each piece separately.

![joinQuery.PNG](joinQuery.PNG "big join")

We'll begin with the **JOIN** (highlighted in blue above). This specifies the sources of data and how to join them. We use **ON** to specify that we combine the tables by matching the values in the `repo_name` columns in the tables.

Next, we'll talk about **SELECT** and **GROUP BY** (highlighted in yellow). The **GROUP BY** breaks the data into a different group for each license, before the **COUNT** the number of rows in the `sample_files` table that corresponds to each license (remember that you can count the number of rows with `COUNT(1)` .)

Finally, the **ORDER BY** (highlighted in purple) sorts the results so that licenses with more files appear first.

It was a big query, but it gave us a nice table summarizing how many files have been committed under each license

# Exercises

## Intro

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

Let's fetch the `stackoverflow` dataset:

In [6]:
from google.cloud import bigquery

# client
client = bigquery.Client()

# db ref
db_ref = client.dataset("stackoverflow", project = "bigquery-public-data")

# API call
db = client.get_dataset(db_ref)

## Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [8]:
[l.table_id for l in client.list_tables(db)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

## Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Fetch the table and look at the first five rows.

In [10]:
# tb ref
tb_ref = db.table("posts_answers")
# tb
tb = client.get_table(tb_ref)

# look at first five rows
client.list_rows(tb, max_results = 5).to_dataframe()

,id,body,comment_count,community_owned_date,creation_date,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags
0,47949879,<p>Building onto what you have nearly got..</p...,0,None,2017-12-23 04:57:17.177000+00:00,2017-12-23 04:57:17.177000+00:00,NaT,,NaN,,5602957,47949810,2,2,
1,15503802,"<p>Normally, ""naturally aligned"" means that an...",1,None,2013-03-19 15:34:54.153000+00:00,2013-03-19 15:34:54.153000+00:00,NaT,,NaN,,179910,15503537,2,3,
2,3497784,"<p>Looking with Reflector, it seems to indicat...",0,None,2010-08-16 22:04:25.970000+00:00,2010-08-16 22:04:25.970000+00:00,NaT,,NaN,,1831,3497732,2,2,
3,39122150,<p>Probably not. I don't know what version of ...,1,None,2016-08-24 11:30:55.667000+00:00,2016-08-24 11:30:55.667000+00:00,2017-05-23 12:09:42.380000+00:00,,-1.0,,2868335,39110336,2,3,
4,36403800,"<p><code>replace(',', '')</code> only replaces...",0,None,2016-04-04 13:18:15.710000+00:00,2016-04-04 13:18:15.710000+00:00,NaT,,NaN,,2825245,36403626,2,2,


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [11]:
# tb ref
tb_ref = db.table("posts_questions")
# tb
tb = client.get_table(tb_ref)

# look at first five rows
client.list_rows(tb, max_results = 5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count
0,9804136,"How do I clear an array of a structure in C, a...",<p>Two questions:</p>\n\n<ol>\n<li>How to quic...,9804289.0,3,2,None,2012-03-21 11:54:26.897000+00:00,NaN,2012-03-21 14:33:12.307000+00:00,2012-03-21 14:33:12.307000+00:00,,882600.0,,662586,1,1,c|struct,6462
1,44896935,How to implement threading to run two bash she...,<p>I have to record a wav file and at the same...,44897223.0,2,6,None,2017-07-04 04:14:50.020000+00:00,1.0,2017-07-11 09:23:08.210000+00:00,2017-07-11 09:23:08.210000+00:00,,3178797.0,,3178797,1,1,python|multithreading|bash|shell|python-multit...,411
2,3059091,How to remove carriage returns from output of ...,<p>I am using wordpress as a CMS and trying to...,3592816.0,7,5,None,2010-06-17 05:26:30.147000+00:00,11.0,2017-05-11 18:05:49.817000+00:00,2010-08-28 06:13:26.817000+00:00,,82330.0,,82330,1,26,php|javascript|wordpress|google-maps,73286
3,8926063,"Code Coverage: Why is end marker red (End If, ...",<p>I use MS-Test with Visual Studio 2010 and V...,8934375.0,4,0,None,2012-01-19 12:32:19.227000+00:00,NaN,2012-01-19 22:33:52.037000+00:00,2012-01-19 13:16:56.100000+00:00,,254041.0,,254041,1,4,vb.net|visual-studio|mstest|code-coverage,566
4,52977342,incorrect checksum for freed object - object w...,<p>I was hoping the return values in c++11 are...,NaN,0,10,None,2018-10-24 20:17:45.643000+00:00,1.0,2018-10-24 20:17:45.643000+00:00,NaT,,NaN,,1935611,1,0,c++|c++11,203


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to. `posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:

determine the `tags` for each answer, and then
select the `owner_user_id` of the answers on the desired tag.
This is exactly what you will do over the next few questions.

## Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [14]:
query = """
        SELECT id, title, owner_user_id, tags
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE tags LIKE "%bigquery%"
        """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
query_out = query_job.to_dataframe()
# check first five rows
query_out.head()


,id,title,owner_user_id,tags
0,45888998,BigQuery GROUP BY function still showing dupli...,8127354.0,sql|duplicates|google-bigquery
1,55590174,Rolling active users that were also active in ...,7635493.0,google-bigquery|window-functions
2,51092204,Multiple Left Joins in BigQuery,8999037.0,sql|join|google-bigquery|left-join|legacy-sql
3,43864176,"I am getting this error in my R code ""Error in...",NaN,mysql|r|google-bigquery
4,51274802,Unable to write multiple partitions in a singl...,8206319.0,java|google-bigquery|standard-sql


## Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [23]:
query = """
        WITH q AS
        (
        SELECT questions.id
        FROM `bigquery-public-data.stackoverflow.posts_questions` as questions
        WHERE questions.tags LIKE "%bigquery%" 
        )
        SELECT a.id AS id, a.body AS body, a.owner_user_id AS owner_user_id
        FROM `bigquery-public-data.stackoverflow.posts_answers` as a
        INNER JOIN q
            ON a.parent_id = q.id        
        """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**11)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
query_out = query_job.to_dataframe()
# check first five rows
query_out.head()

,id,body,owner_user_id
0,56068280,<p>HTTP based sources for Cloud Data Fusion ar...,3565788.0
1,44252795,<p>Without knowing the detail of your scenario...,7459398.0
2,11909802,<p>I don't think the BigQuery tool currently h...,1362175.0
3,50688711,<p>Use can use both:</p>\n\n<pre><code>bq quer...,2504477.0
4,38150592,<p>After you link your Firebase app to BigQuer...,5385206.0


## Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [25]:
query = """
        WITH q AS
        (
        SELECT questions.id
        FROM `bigquery-public-data.stackoverflow.posts_questions` as questions
        WHERE questions.tags LIKE "%bigquery%" 
        )
        SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
        FROM `bigquery-public-data.stackoverflow.posts_answers` as a
        INNER JOIN q
            ON a.parent_id = q.id
        GROUP BY user_id
        ORDER BY number_of_answers DESC
        """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**11)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
query_out = query_job.to_dataframe()
# check first five rows
query_out.head()

,user_id,number_of_answers
0,5221944.0,2338
1,6253347.0,698
2,132438.0,687
3,1366527.0,617
4,243782.0,450


## Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Check the solution below.

In [40]:
def expert_finder(topic, client): # prob an argument for the maximum bytes would be handy
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like "%{}%"
               GROUP BY a.owner_user_id
               ORDER BY number_of_answers DESC
               """.format(topic)
    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**11)      
    my_query_job = client.query(my_query, job_config = safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results

In [41]:
expert_finder("bigquery", client).head()

,user_id,number_of_answers
0,5221944.0,2338
1,6253347.0,698
2,132438.0,687
3,1366527.0,617
4,243782.0,450


In [42]:
expert_finder("python", client).head()

,user_id,number_of_answers
0,100297.0,19004
1,2901002.0,14020
2,NaN,13397
3,104349.0,8143
4,190597.0,7083
